In [235]:
import pandas as pd
import cv2
import os
import random
import math
from tqdm import tqdm

In [236]:
video_path = "/home/theo/Documents/Unif/Master/ChimpRec/ChimpRec-Dataset/CCR/videos"
annotations_path = "/home/theo/Documents/Unif/Master/ChimpRec/ChimpRec-Dataset/CCR/metadata/annotations/body_data.csv"
dataset_path = "/home/theo/Documents/Unif/Master/ChimpRec/ChimpRec-Dataset/Chimpanzee_detection_dataset"

In [237]:
df = pd.read_csv(annotations_path)
print(df.loc[df["year"] == 2013])

         year   video  frame  track         x         y         w         h  \
1316517  2013  13.mp4      1      0  0.680208  0.072917  0.231250  0.450000   
1316518  2013  13.mp4      1      2  0.405208  0.137500  0.210417  0.568519   
1316519  2013  13.mp4      1      1  0.216667  0.042708  0.255208  0.601852   
1316520  2013  13.mp4      2      0  0.679167  0.067708  0.230208  0.457407   
1316521  2013  13.mp4      2      2  0.404167  0.137500  0.209375  0.559259   
...       ...     ...    ...    ...       ...       ...       ...       ...   
1937580  2013  10.mp4  11170    649  0.566667  0.182292  0.182292  0.294444   
1937581  2013  10.mp4  11171    649  0.560417  0.180208  0.186458  0.298148   
1937582  2013  10.mp4  11171    579  0.340625  0.121875  0.225000  0.435185   
1937583  2013  10.mp4  11172    579  0.337500  0.120833  0.226042  0.440741   
1937584  2013  10.mp4  11172    649  0.557292  0.167708  0.192708  0.320370   

           label  
1316517  NOTSURE  
1316518    FA

In [238]:
proportion_kept = 0.0015 # approximately 1500 images

In [239]:
# @input: n: size of the sample
# proportion: (proportion*100)% of the numbers in [0, n-1]
# @outputs
# No numbers in common between the outputs
def segment_dataset(n, proportion):
    # Calculate the number of elements to select based on the proportion
    num_elements = int(n * proportion)
    
    # Generate evenly spaced numbers within the range [0, n-1]
    step = n / num_elements
    selected_numbers = [int(i * step) for i in range(num_elements)]
    
    return selected_numbers

In [240]:
def get_len_video(path_to_video):
    video = cv2.VideoCapture(f"{video_path}/{path_to_video}")

    if not video.isOpened():
        print(f"Could not open video: {path_to_video}")
        return 0
    
    length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    video.release()
    return length

In [241]:
def extract_metadata(video_type, video_id, frame_number):
    sample_id = video_id.replace(".mp4", "")
    sub_df = df.loc[(df["video"] == video_id) & (df["frame"] == frame_number)]
    output_file = f"{dataset_path}/labels/{video_type}/{sample_id}_{frame_number}.txt"

    string_output = ""

    for index, row in sub_df.iterrows():
        string_output += f"\n0 {row['x']} {row['y']} {row['w']} {row['h']}"

    with open(output_file, "w") as file:
        file.write(string_output.strip())
    file.close()
    
    


In [242]:
# @input:
# type: "train", "val" or "test" --> indicates which part of the dataset it is
# sample_id: name of the video with the extension
# idxs: indexes of the images to keep
# @output:
# nothing, the image are saved in the output folder
def extract_data(type, video_id):

    sample_id = video_id[:-4]

    output_folder = f"{dataset_path}/images/{type}"
    video = cv2.VideoCapture(f"{video_path}/{video_id}")
    len_video = get_len_video(video_id)

    idxs = segment_dataset(len_video, proportion_kept)
    idxs = sorted(idxs)

    for frame_count in idxs:

        # extract the annotations
        extract_metadata(type, video_id, frame_count)

        # Set the video to the specific frame
        video.set(cv2.CAP_PROP_POS_FRAMES, frame_count)
        success, frame = video.read()
        
        if success:
            frame_filename = f"{output_folder}/{sample_id}_{frame_count}.jpg"
            cv2.imwrite(frame_filename, frame)
        else:
            print(f"Could not read frame {frame_count} in video {video_id}")

    # Release the video capture object
    video.release()

In [243]:
filenames = os.listdir(video_path)
n_files = len(filenames)

train_prop = 0.85

numbers = list(range(n_files))
random.shuffle(numbers)

train_video_idxs = numbers[:math.ceil(train_prop*n_files)]
val_video_idxs = numbers[math.ceil(train_prop*n_files):]

def process_dataset(video_idxs, type):
    # progression bar added
    for video_idx in tqdm(video_idxs, desc=f"Processing videos: {type}", colour="green"):
        video_id = filenames[video_idx]
        extract_data(type, video_id)
        # print(df.loc[df["video"] == sample_id])

        

In [244]:
process_dataset(train_video_idxs, "train")
process_dataset(val_video_idxs, "val")

Processing videos: val: 100%|██████████| 4/4 [00:14<00:00,  3.71s/it]
